# Mediator Pattern

## Intent
Define an object that encapsulates how a set of objects interact. Mediator promotes loose coupling by keeping objects from referring to each other explicitly.

## Problem
You have many objects that need to communicate but:
- Direct communication creates tight coupling
- Objects reference each other in complex ways
- Hard to understand and maintain interactions
- Difficult to change communication logic

**Real-world analogy**: Air traffic control tower - planes don't talk to each other directly, they communicate through the tower

## When to Use
✅ **Use when:**
- Objects communicate in complex but well-defined ways
- Reusing objects is difficult due to many dependencies
- Behavior distributed among multiple classes
- Want to customize behavior without subclassing

❌ **Avoid when:**
- Simple one-to-one communication
- Objects don't interact much
- Mediator becomes too complex (God object)

## Pattern Structure
```
┌─────────┐         ┌──────────┐
│Mediator │◄────────│Colleague │
├─────────┤         ├──────────┤
│notify() │         │mediator  │
└─────────┘         │send()    │
     ▲              └──────────┘
     │                    ▲
┌────┴──────┐        ┌───┴────┐
│Concrete   │        │Concrete│
│Mediator   │        │Colleagu│
├───────────┤        └────────┘
│colleagues │
└───────────┘
```

## Example 1: Without Mediator

**Problem**: Objects tightly coupled, complex references

In [ ]:
# WITHOUT Mediator - Tight coupling
class User:
    def __init__(self, name):
        self.name = name
        self.contacts = []  # Direct references!
    
    def add_contact(self, user):
        self.contacts.append(user)
    
    def send_message(self, message):
        # Sends to all contacts directly
        for contact in self.contacts:
            contact.receive(message, self.name)
    
    def receive(self, message, from_user):
        print(f"{self.name} received from {from_user}: {message}")

# Create users
alice = User("Alice")
bob = User("Bob")
charlie = User("Charlie")

# Set up connections (messy!)
alice.add_contact(bob)
alice.add_contact(charlie)
bob.add_contact(alice)
bob.add_contact(charlie)
charlie.add_contact(alice)
charlie.add_contact(bob)

print("\n❌ Tight coupling! Each user knows about all others!")
print("❌ Complex web of references!")
print("❌ Hard to change communication logic!")

## Implementation: Mediator Pattern

In [ ]:
from abc import ABC, abstractmethod
from typing import List

# Mediator interface
class ChatMediator(ABC):
    """Abstract mediator for chat."""
    
    @abstractmethod
    def send_message(self, message: str, user: 'User') -> None:
        pass
    
    @abstractmethod
    def add_user(self, user: 'User') -> None:
        pass


# Concrete mediator
class ChatRoom(ChatMediator):
    """Concrete mediator - chat room."""
    
    def __init__(self, name: str):
        self.name = name
        self.users: List[User] = []
    
    def add_user(self, user: 'User') -> None:
        self.users.append(user)
        print(f"  👤 {user.name} joined {self.name}")
    
    def send_message(self, message: str, sender: 'User') -> None:
        """Broadcast message to all users except sender."""
        print(f"  📢 [{self.name}] {sender.name}: {message}")
        for user in self.users:
            if user != sender:
                user.receive(message, sender.name)


# Colleague
class User:
    """User - communicates through mediator."""
    
    def __init__(self, name: str, mediator: ChatMediator):
        self.name = name
        self.mediator = mediator  # Reference to mediator only!
        mediator.add_user(self)
    
    def send(self, message: str) -> None:
        """Send message through mediator."""
        self.mediator.send_message(message, self)
    
    def receive(self, message: str, from_user: str) -> None:
        """Receive message from mediator."""
        print(f"    💬 {self.name} received from {from_user}: {message}")


# Demo
print("\n=== Mediator Pattern (Chat Room) ===")

print("\n1. Creating chat room:")
general = ChatRoom("#general")

print("\n2. Users joining:")
alice = User("Alice", general)
bob = User("Bob", general)
charlie = User("Charlie", general)

print("\n3. Sending messages:")
alice.send("Hello everyone!")
print()
bob.send("Hi Alice!")
print()
charlie.send("Hey team!")

print("\n✅ Users don't know about each other!")
print("✅ All communication through mediator!")

## Real-World Example: Air Traffic Control

In [ ]:
from typing import Optional

# Mediator
class ControlTower:
    """Air traffic control tower - mediator."""
    
    def __init__(self, name: str):
        self.name = name
        self.aircrafts = []
        self.runway_available = True
    
    def register_aircraft(self, aircraft: 'Aircraft') -> None:
        self.aircrafts.append(aircraft)
        print(f"  📡 {aircraft.call_sign} registered with {self.name}")
    
    def request_landing(self, aircraft: 'Aircraft') -> bool:
        """Aircraft requests landing clearance."""
        print(f"  📻 {aircraft.call_sign}: Requesting landing clearance")
        
        if self.runway_available:
            self.runway_available = False
            print(f"  ✅ {self.name}: {aircraft.call_sign}, you are clear to land")
            return True
        else:
            print(f"  ⏳ {self.name}: {aircraft.call_sign}, hold pattern, runway occupied")
            return False
    
    def request_takeoff(self, aircraft: 'Aircraft') -> bool:
        """Aircraft requests takeoff clearance."""
        print(f"  📻 {aircraft.call_sign}: Requesting takeoff clearance")
        
        if self.runway_available:
            self.runway_available = False
            print(f"  ✅ {self.name}: {aircraft.call_sign}, you are clear for takeoff")
            return True
        else:
            print(f"  ⏳ {self.name}: {aircraft.call_sign}, hold position, runway in use")
            return False
    
    def notify_runway_clear(self, aircraft: 'Aircraft') -> None:
        """Aircraft notifies runway is clear."""
        print(f"  📻 {aircraft.call_sign}: Runway clear")
        self.runway_available = True
        print(f"  ✅ {self.name}: Roger, runway available")


# Colleague
class Aircraft:
    """Aircraft - communicates through control tower."""
    
    def __init__(self, call_sign: str, tower: ControlTower):
        self.call_sign = call_sign
        self.tower = tower  # Only knows about mediator
        tower.register_aircraft(self)
    
    def land(self) -> None:
        """Attempt to land."""
        if self.tower.request_landing(self):
            print(f"  ✈️  {self.call_sign}: Landing...")
            self.tower.notify_runway_clear(self)
    
    def takeoff(self) -> None:
        """Attempt to take off."""
        if self.tower.request_takeoff(self):
            print(f"  🛫 {self.call_sign}: Taking off...")
            self.tower.notify_runway_clear(self)


# Demo
print("\n=== Air Traffic Control Mediator ===")

print("\n1. Setting up control tower:")
tower = ControlTower("JFK Tower")

print("\n2. Registering aircraft:")
flight1 = Aircraft("AA123", tower)
flight2 = Aircraft("UA456", tower)
flight3 = Aircraft("DL789", tower)

print("\n3. Flight operations:")
print("\n   Flight 1 landing:")
flight1.land()

print("\n   Flight 2 tries to take off (runway busy):")
flight2.takeoff()

print("\n   Flight 2 tries again (runway now clear):")
flight2.takeoff()

print("\n   Flight 3 landing:")
flight3.land()

print("\n✅ Aircraft communicate through tower, not directly!")

## Real-World Example: Smart Home System

In [ ]:
# Mediator
class SmartHomeHub:
    """Smart home hub - coordinates devices."""
    
    def __init__(self):
        self.devices = {}
        self.mode = "normal"  # normal, away, sleep
    
    def register_device(self, name: str, device: 'SmartDevice') -> None:
        self.devices[name] = device
        print(f"  🔌 {name} connected to hub")
    
    def set_mode(self, mode: str) -> None:
        """Change home mode - coordinates all devices."""
        print(f"\n  🏠 Setting home mode: {mode}")
        self.mode = mode
        
        if mode == "away":
            # Turn off most devices
            self.devices["lights"].turn_off()
            self.devices["thermostat"].set_temperature(60)
            self.devices["security"].activate()
            print("  🔒 Away mode activated")
        
        elif mode == "sleep":
            # Dim lights, lower temp, activate security
            self.devices["lights"].dim(10)
            self.devices["thermostat"].set_temperature(68)
            self.devices["security"].activate()
            print("  😴 Sleep mode activated")
        
        elif mode == "normal":
            # Restore normal settings
            self.devices["lights"].turn_on()
            self.devices["thermostat"].set_temperature(72)
            self.devices["security"].deactivate()
            print("  ☀️  Normal mode activated")
    
    def notify(self, event: str, device_name: str) -> None:
        """Handle device events."""
        if event == "motion_detected" and self.mode == "away":
            print(f"  ⚠️  Motion detected by {device_name}!")
            self.devices["lights"].turn_on()
            print("  🚨 Activating alarm!")


# Base device
class SmartDevice:
    """Base smart device."""
    
    def __init__(self, name: str, hub: SmartHomeHub):
        self.name = name
        self.hub = hub  # Reference to mediator


# Concrete devices
class SmartLights(SmartDevice):
    def __init__(self, hub: SmartHomeHub):
        super().__init__("lights", hub)
        self.brightness = 100
    
    def turn_on(self) -> None:
        self.brightness = 100
        print(f"    💡 Lights: ON (100%)")
    
    def turn_off(self) -> None:
        self.brightness = 0
        print(f"    💡 Lights: OFF")
    
    def dim(self, brightness: int) -> None:
        self.brightness = brightness
        print(f"    💡 Lights: Dimmed to {brightness}%")


class SmartThermostat(SmartDevice):
    def __init__(self, hub: SmartHomeHub):
        super().__init__("thermostat", hub)
        self.temperature = 72
    
    def set_temperature(self, temp: int) -> None:
        self.temperature = temp
        print(f"    🌡️  Thermostat: Set to {temp}°F")


class SecuritySystem(SmartDevice):
    def __init__(self, hub: SmartHomeHub):
        super().__init__("security", hub)
        self.active = False
    
    def activate(self) -> None:
        self.active = True
        print(f"    🔐 Security: Armed")
    
    def deactivate(self) -> None:
        self.active = False
        print(f"    🔐 Security: Disarmed")
    
    def detect_motion(self) -> None:
        if self.active:
            self.hub.notify("motion_detected", self.name)


# Demo
print("\n=== Smart Home Mediator ===")

print("\n1. Setting up smart home:")
hub = SmartHomeHub()

lights = SmartLights(hub)
thermostat = SmartThermostat(hub)
security = SecuritySystem(hub)

hub.register_device("lights", lights)
hub.register_device("thermostat", thermostat)
hub.register_device("security", security)

print("\n2. Changing modes:")
hub.set_mode("sleep")
hub.set_mode("away")

print("\n3. Motion detected while away:")
security.detect_motion()

print("\n4. Returning home:")
hub.set_mode("normal")

print("\n✅ Hub coordinates all devices!")

## Advantages & Disadvantages

### ✅ Advantages
1. **Loose coupling**: Objects don't reference each other directly
2. **Centralized control**: Communication logic in one place
3. **Reusability**: Colleagues don't depend on each other
4. **Simplified protocols**: Objects only know mediator interface
5. **Easy to add colleagues**: Extend without modifying existing code

### ❌ Disadvantages
1. **Complexity**: Mediator can become complex (God object)
2. **Single point of failure**: Mediator failure affects all
3. **Performance**: Extra indirection layer
4. **Hard to maintain**: Complex mediator is hard to understand

## Mediator vs Observer

**Mediator**:
- Coordinates communication between objects
- Objects know about mediator
- Bi-directional communication
- Mediator encapsulates interaction logic

**Observer**:
- One-to-many notification
- Subject doesn't know observers
- Uni-directional communication
- Subject broadcasts to all observers

## Common Use Cases

1. **Chat rooms**: Users communicate through chat room
2. **Air traffic control**: Planes coordinate through tower
3. **GUI dialogs**: Components interact through dialog
4. **Smart home**: Devices coordinate through hub
5. **MVC**: Controller acts as mediator
6. **Game networking**: Players communicate through server

## Related Patterns

- **Facade**: Simplifies interface (one-way), Mediator coordinates (two-way)
- **Observer**: Can implement mediator with observer
- **Chain of Responsibility**: Linear chain vs mediator hub

## Best Practices

1. **Keep mediator focused**: Don't make it a God object
2. **Define clear protocols**: Document communication patterns
3. **Avoid business logic**: Keep in colleagues when possible
4. **Use interfaces**: Define mediator and colleague interfaces
5. **Consider observer**: For simple broadcast patterns
6. **Limit colleague knowledge**: Colleagues should only know mediator interface

## Summary

Mediator pattern enables:
- Loose coupling between objects
- Centralized communication logic
- Simplified object protocols
- Easy to modify interactions

Perfect for: Chat systems, traffic control, UI dialogs, smart homes, game networking.

**Key Insight**: Encapsulate object interactions in a mediator, preventing direct references and reducing coupling!